# Tutorial for one-photon imaging dataset.

This tutorial shows how to access the *Two Photon dataset* using `pynwb`. 

This dataset contains the 2-photon calcium imaging holographic stimulation data and visual stimuli events.

Contents:

- [Reading an NWB file](#read-nwb)
- [Access subject and task metadata](#access-subject)
- [Access Imaging](#access-imaging)
- [Access TTL Signals](#ttl-signals)
- [Access Wheel Signal](#wheel-signal)
- [Access Raw Behavior](#raw-behavior)
- [Access Processed Behavior](#processed-behavior)
- [Access Visual Stimulus](#access-visualstim)

A schematic representation where the source data is saved in NWB:

![Alt text](./conversion_outline_diagram.png)

# Reading an NWB file <a name="read-nwb"></a>

This section demonstrates how to read an NWB file using `pynwb`.

Based on the [NWB File Basics](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#sphx-glr-tutorials-general-plot-file-py) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

Reading is carried out using the [NWBHDF5IO](https://pynwb.readthedocs.io/en/stable/pynwb.html#pynwb.NWBHDF5IO) class. To read the NWB file use the read mode ("r") to retrieve an NWBFile object.


In [ ]:
from pynwb import NWBHDF5IO
from pathlib import Path

# Choose which epoch to explore
session_id = '11222019_grabAM06_vis_stim'

# The file path to a .nwb file
root_path = Path("/media/amtra/Samsung_T5/CN_data")
output_dir_path = root_path / "Higley-conversion_nwb/nwb_stub/"
nwbfile_path = output_dir_path / f"{session_id}.nwb"
io = NWBHDF5IO(path=nwbfile_path, mode="r", load_namespaces=True)
nwbfile = io.read()

nwbfile

Importantly, the `session start time` is the reference time for all timestamps in the file. For instance, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time.

The `session_start_time` is extracted from the ScanImage metadata (`epoch`) of the first .tiff of the epoch.

In [ ]:
nwbfile.session_start_time

# Access subject metadata <a name="access-subject"></a>

This section demonstrates how to access the [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field in an NWB file.

The [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field can be accessed as `nwbfile.subject`.


In [ ]:
nwbfile.subject

# Access OnePhoton Imaging <a name="access-imaging"></a>

This section demonstraces how to access the raw Two Photon imaging data.

`NWB` organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWBFile and the types of data they are intended to store:

- `acquisition`: raw, acquired data that should never change
- `processing`: processed data, typically the results of preprocessing algorithms and could change

## Raw OnePhoton Imaging

The raw TwoPhoton imaging data is stored in `pynwb.ophys.OnePhotonSeries` objects (for each channel and excitation type separately) which is added to `nwbfile.acquisition`. The data can be accessed as `nwbfile.acquisition['OnePhotonSeries_color_Excitation_color_Channel']`.

The data in [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

In [ ]:
names_of_photon_series = nwbfile.acquisition.keys()
_ = [print(name) for name in names_of_photon_series if "OnePhotonSeries" in name ]

In [ ]:
excitation_channel_combination = "BlueExcitationGreenChannel"

In [ ]:
photon_series = nwbfile.acquisition[f"OnePhotonSeries{excitation_channel_combination}"]

In [ ]:
photon_series.imaging_plane.indicator

In [ ]:
# Visualize the imaging data.

from matplotlib import pyplot as plt

plt.imshow(photon_series.data[10], aspect="auto", cmap="RdYlBu_r")
plt.title(f"One frame from {excitation_channel_combination}")
plt.show()


In [ ]:
photon_series.rate

In [ ]:
photon_series.starting_time

# Access TTL Signals <a name="#ttl-signals"></a>

This section demonstrates how to access the TTL signals stored in the NWBFile.
TTLTypesTable contains the description and the id for each TTL signal
TTLsTable contains the respective timestamps (rising times)

In [ ]:
nwbfile.acquisition["TTLTypesTable"]

In [ ]:
nwbfile.acquisition["TTLsTable"]


# Access Wheel Signal <a name="#wheel-signal"></a>

This section demonstrates how to access the wheel velocity trace stored in the NWBFile.

In [ ]:
wheel_signal = nwbfile.acquisition["WheelSignal"]
wheel_signal

In [ ]:
import numpy as np
time = np.linspace(0, wheel_signal.data.shape[0]/wheel_signal.rate,wheel_signal.data.shape[0])
plt.plot(time,wheel_signal.data*wheel_signal.conversion)
plt.ylabel(wheel_signal.unit)
plt.xlabel("Time (s)")
plt.title(wheel_signal.name)

# Access Raw Behavior <a name="#raw-behavior"></a>

This section demonstrates how to access the raw behavioral video stored in the NWBFile.

In [ ]:
video = nwbfile.acquisition["Video: 11222019_grabAM06_visual_stim"]
video

In [ ]:
plt.imshow(video.data[1], aspect="auto")
plt.title(f"A frame of {video.name} at {video.timestamps[1]} sec")
plt.show()

# Access Processed Behavior <a name="#processed-behavior"></a>

This section demonstrates how to access the Facemap output stored in the NWBFile.


In [ ]:
nwbfile.processing["behavior"]["EyeTracking"]

In [ ]:
nwbfile.processing["behavior"]["PupilTracking"]

In [ ]:
motion_svd_series = nwbfile.processing["behavior"]["MotionSVDSeriesROI1"]
motion_svd_series

In [ ]:
motion_svd_series.data.shape

In [ ]:
plt.plot(motion_svd_series.data[0])
plt.title(motion_svd_series.name)

In [ ]:
motion_svd_masks = nwbfile.processing["behavior"]["MotionSVDMasksROI1"]
motion_svd_masks

In [ ]:
motion_svd_masks.mask_coordinates[:]

In [ ]:
motion_svd_masks.processed_frame_dimension[:]

In [ ]:
masks = motion_svd_masks.image_mask_index[:10]
# plt.imshow(mask, aspect="auto", cmap="RdYlBu_r" )
# plt.title("First component mask")
# plt.show()

# Access Visual Stimulus <a name="access-visualstim"></a>

This section demonstrates how to access the visual stimulus data.

In [ ]:
nwbfile.intervals["VisualStimulus"]